In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.impute import KNNImputer

In [13]:
df = pd.read_csv('../Data/df_transformed.csv')

In [14]:
df.drop('Unnamed: 0',axis=1,inplace=True)

# First just looking at demographic variables, original nutrition variables, physical activity and blood pressure and pulse

In [15]:
numerical_variables = ['RIAGENDR','RIDAGEYR','BMXBMI','BPXPLS','DR1TKCAL', 'DR1TPROT', 'DR1TCARB', 'DR1TSUGR',
       'DR1TFIBE', 'DR1TTFAT', 'DR1TSFAT', 'DR1TMFAT', 'DR1TPFAT', 'DR1TCHOL',
       'DR1TSODI', 'DR1TVD', 'DR1TCALC', 'DR1TIRON', 'DR1TPOTA','MET_rec','Systolic','Diastolic']
categorical_variables = ['RIDRETH3','DR1_300']

In [16]:
df1 = df[numerical_variables+categorical_variables+['LBXTC']].copy()

In [17]:
df1.dropna(inplace=True)

In [18]:
y = df1.LBXTC
X = df1.drop('LBXTC', axis=1)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=625)

In [20]:
# maybe train a regular random forest then add info and keep training a regular randome forest until you get one that is 
# ok, then do hyperparameter tuning.